# Interacting with Assets in VAM API

This notebook provides a comprehensive guide on how to interact with different types of assets using the VAM API.
We will cover:
- Querying different asset types
- Filtering assets by execution venue
- Working with different asset types like spot, futures, and equity
- Performing batch operations

For this example, we will use the following assets:
- **Binance Spot**: `BTCUSDT`, `ETHUSDT`
- **Binance Futures**: `BTCUSDT`, `ETHUSDT`
- **Alpaca Equity**: `NVDA`, `MSFT`

We will demonstrate how to query assets across these execution venues.


In [1]:

import dotenv
dotenv.load_dotenv('../../../.env')
import mainsequence    
from mainsequence.client import Asset, AssetFutureUSDM, AssetCurrencyPair,AssetCategory
from mainsequence.client import MARKETS_CONSTANTS

# Define execution venue symbols
BINANCE_EV = MARKETS_CONSTANTS.BINANCE_EV_SYMBOL
BINANCE_FUTURES_EV = MARKETS_CONSTANTS.BINANCE_FUTURES_EV_SYMBOL
ALPACA_EV = MARKETS_CONSTANTS.ALPACA_EV_SYMBOL

## Querying All Assets

To get all assets available in the system, we can use the `Asset.filter()` method.


## Optimal Asset Queries

There are many assets, and there will always be more. We recommend narrowing your search so that the expected results are in the hundreds. If you need more assets, it is best to first create a category that encompasses those assets and then query by that category.

A good way to start narrowing asset searches is by filtering on the execution venue and some FIGI properties. For example, if you want to retrieve the equivalent assets from one category on another exchange, you could do something like this:



In [2]:
top_100_cryptos = AssetCategory.get(unique_identifier="top_100_crypto_market_cap")

# Switch to Binance categories
spot_assets = Asset.filter(id__in=top_100_cryptos.assets)

# Get them through the main sequence FIGI class and exchange
binance_currency_pairs = AssetCurrencyPair.filter(
    base_asset__figi_details__main_sequence_share_class__in=[
        a.figi_details.main_sequence_share_class for a in spot_assets
    ],
    execution_venue__symbol=MARKETS_CONSTANTS.BINANCE_EV_SYMBOL,
    quote_asset__symbol="USDT",
    include_base_quote_detail=False
)



## Filtering Assets by Type

Different asset types have distinct properties, which can help you extend your filters. It's important to note that all asset types inherit from the Asset class. Therefore, you can always use the Asset class to access all assets and their general properties.

In [3]:
# Filtering Simple Assets
simple_assets = Asset.filter(symbol="BTCUSDT")
print(f"Total Spot Crypto Assets on Binance: {len(simple_assets)}")

# Filtering Futures
cash_equity_assets = AssetCurrencyPair.filter(symbol="BTCUSDT")
print(f"Total Assets CurrencyPair Assets : {len(cash_equity_assets)}")

# Filtering  Futures USDM Assets
futures_assets= AssetFutureUSDM.filter(symbol="BTCUSDT")
print(f"Total Futures Assets : {len(futures_assets)}")




Total Spot Crypto Assets on Binance: 5
Total Assets CurrencyPair Assets : 3
Total Futures Assets : 2


[Asset: 192404, Asset: 200468, Asset: 200470, Asset: 192403, Asset: 200469]

[Asset: 192404, Asset: 200468, Asset: 200470, Asset: 192403, Asset: 200469]

## Using `filter_with_asset_class`

The `filter_with_asset_class` method is an extended version of the `filter` method.
It ensures that each returned asset is an instance of its correct class based on its `asset_type`.

### When to Use?
- When querying multiple asset types and needing them returned with the correct class.
- When working with assets across execution venues and requiring proper type differentiation.

### Example 1: Query Binance Spot Assets with Correct Classes


In [9]:
# Query Binance Spot assets with correct asset classes
binance_assets = Asset.filter_with_asset_class(symbol="BTCUSDT")

# Displaying asset information
for asset in binance_assets:
    print(f"Asset: {asset.symbol}, Type: {asset.asset_type}, Class: {type(asset).__name__}")


ValidationError: 2 validation errors for BinanceCurrencyPair
base_asset
  Field required [type=missing, input_value={'id': 11489, 'symbol': '...'unique_symbol': 'LINK'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
quote_asset
  Field required [type=missing, input_value={'id': 11489, 'symbol': '...'unique_symbol': 'LINK'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

### Example 2: Query Binance Futures and Alpaca Equity Assets

This example demonstrates how the method ensures each asset is returned with its appropriate class.


In [ ]:
# Query Binance Futures and Alpaca Equity Assets with correct classes
binance_futures_assets = Asset.filter_with_asset_class(execution_venue__symbol=CONSTANTS.BINANCE_FUTURES_EV_SYMBOL)
alpaca_equity_assets = Asset.filter_with_asset_class(execution_venue__symbol=CONSTANTS.ALPACA_EV_SYMBOL)

# Displaying results
for asset in binance_futures_assets + alpaca_equity_assets:
    print(f"Asset: {asset.symbol}, Type: {asset.asset_type}, Class: {type(asset).__name__}")